In [1]:
import os
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Sail2304/Ola-driver-churn.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Sail2304"
os.environ["MLFLOW_TRACKING_PASSWORD"]="b9b3f722c5b2cfcfba8e769fcd6c4ffd37e6136b"

In [2]:
%pwd

'd:\\Scaler Course\\OLACaseStudy\\Ola-driver-churn\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Scaler Course\\OLACaseStudy\\Ola-driver-churn'

In [24]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    ohencoder_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [6]:
from src.OLAChurnPred.constants import *
from src.OLAChurnPred.utils.common import read_yaml, create_directories,save_json

In [25]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self)->ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.Final_Params
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            ohencoder_path=config.ohencoder_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/Sail2304/Ola-driver-churn.mlflow"
        )

        return model_evaluation_config

In [8]:
import os 
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [22]:
class ModelEvaluation():
    def __init__(self, config:ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self, actual, pred):
        accuracy = accuracy_score(actual, pred)
        precision = precision_score(actual,pred)
        recall = recall_score(actual,pred)
        f1 = f1_score(actual, pred)
        cm = confusion_matrix(actual,pred)

        return accuracy, precision, recall, f1, cm
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        X_test = test_data.drop(columns=[self.config.target_column])
        y_test = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            pred = model.predict(X_test)
            (accuracy, precision, recall, f1, cm) = self.eval_metrics(y_test, pred)
            TN, FP, FN, TP = cm.ravel()
            # saving metrics as local
            scores = {"accuracy": accuracy, 
                      "precision":precision, 
                      "recall": recall, 
                      "f1": f1,
                      "TN": int(TN),
                      "FP": int(FP),
                      "FN": int(FN),
                      "TP": int(TP)
            }

            
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("f1", f1)

            mlflow.log_metric("TN", TN)
            mlflow.log_metric("FP", FP)
            mlflow.log_metric("FN", FN)
            mlflow.log_metric("TP", TP)
            mlflow.log_artifact(self.config.ohencoder_path)

            #model registery does not work with file store
            if tracking_url_type_store != "file":
                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="Gradient Boosting Model")
            else:
                mlflow.sklearn.log_model(model, "model")
    



            




In [27]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2024-12-12 22:47:58,087: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-12 22:47:58,101: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-12 22:47:58,107: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-12 22:47:58,107: INFO: common: created directory at: artifacts]
[2024-12-12 22:47:58,119: INFO: common: created directory at: artifacts/model_evaluation]
[2024-12-12 22:47:59,007: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


2024/12/12 22:48:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'Gradient Boosting Model' already exists. Creating a new version of this model...
2024/12/12 22:48:13 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Gradient Boosting Model, version 4
Created version '4' of model 'Gradient Boosting Model'.


🏃 View run caring-bug-429 at: https://dagshub.com/Sail2304/Ola-driver-churn.mlflow/#/experiments/0/runs/563a44497d0e4ae586928517829df980
🧪 View experiment at: https://dagshub.com/Sail2304/Ola-driver-churn.mlflow/#/experiments/0
